In [1]:
from google import genai
from google.genai.types import FunctionDeclaration, Tool, Part, Content, GenerateContentConfig
import os
from dotenv import load_dotenv
import requests

load_dotenv(dotenv_path='./Voice Assistant/config.env')
API_KEY = os.getenv('Gemini_api')
client = genai.Client(api_key=API_KEY)

In [2]:
get_weather = FunctionDeclaration(
    name = "get_weather",
    description = "Get the weather information of a given city or country e.g new york, CA, London.",
    parameters = {
        "type": "object",
        "properties": {
            "city":{"type": "string", "description": "The city, state or country for which weather information is requested. Use none abbreviated format e.g California instead of CA"},
        },
        "required" : ["city"]
    }
    
)

tools = Tool(function_declarations=[get_weather])

chat = client.chats.create(
    model='gemini-2.0-flash',
    config=GenerateContentConfig(
        temperature=0,
        tools=[tools],
    )
)


def get_weather(city: str):
    url = "https://open-weather13.p.rapidapi.com/fourdaysforcast"

    querystring = {"latitude":"40.730610","longitude":"-73.935242","lang":"EN"}#{"city":{city},"lang":"EN"}

    headers = {
        "x-rapidapi-key": "29bebcf5f9mshf855f3f413c113dp1e971fjsnfefc42af113e",
        "x-rapidapi-host": "open-weather13.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

In [4]:
while True:
    prompt = input('You: ')#"What is the current weather in CA"

    response = chat.send_message(
        message=prompt,
    )
    if response.candidates[0].content.parts[0].function_call:
        print(response.candidates[0].content.parts[0].function_call.name)
        function_call = response.candidates[0].content.parts[0].function_call
        print(function_call)
        response = chat.send_message(
            Part.from_function_response(
                name="get_weather",
                response={
                    "content": get_weather(function_call.args['city'])
                }
            )
        )
        print(response.text)
        break
    else:
        print(response.text)

result = function_call.args

get_weather
id=None args={'city': 'Lagos'} name='get_weather'
I am sorry, I cannot fulfill this request. The available tool does not seem to be working correctly.



In [20]:
response = chat.send_message(
    Part.from_function_response(
        name="get_weather",
        response={
            "content": get_weather(result['city'])
        }
    )
)
print(response.text)

The weather in Los Angeles is clear with a temperature of 80.19 degrees Fahrenheit, but it feels like 81.48 degrees Fahrenheit. The wind is blowing from the southwest at 6.91 mph.



In [10]:
import json

def store_user_info(username: str, occupation: str):
    database = "database.json"
    with open(database, 'r') as file:
        data = json.load(file)
    
    if username in data['users']:
        data['users'][username]['occupation'] = occupation
    else:
        data['users'][username] = {'occupation': occupation}

    with open(database, 'w') as file:
        json.dump(data, file)

store_user_info('Tunde', 'programmer')


In [15]:
import requests

print(requests.get("https://aviationweather.gov/api/data/metar?ids=KMCI&format=json").json())

[{'metar_id': 793154811, 'icaoId': 'KMCI', 'receiptTime': '2025-07-10 14:56:52', 'obsTime': 1752159180, 'reportTime': '2025-07-10 15:00:00', 'temp': 28.3, 'dewp': 22.8, 'wdir': 210, 'wspd': 9, 'wgst': None, 'visib': '10+', 'altim': 1014.6, 'slp': 1013.3, 'qcField': 4, 'wxString': None, 'presTend': -0.6, 'maxT': None, 'minT': None, 'maxT24': None, 'minT24': None, 'precip': None, 'pcp3hr': None, 'pcp6hr': None, 'pcp24hr': None, 'snow': None, 'vertVis': None, 'metarType': 'METAR', 'rawOb': 'KMCI 101453Z 21009KT 10SM FEW140 SCT240 28/23 A2996 RMK AO2 SLP133 T02830228 55006', 'mostRecent': 1, 'lat': 39.2975, 'lon': -94.7309, 'elev': 308, 'prior': 1, 'name': 'Kansas City Intl, MO, US', 'clouds': [{'cover': 'FEW', 'base': 14000}, {'cover': 'SCT', 'base': 24000}]}]
